In [ ]:
# Configuration
DELTA_TABLE_PATH = "abfss://datacare@onelake.dfs.fabric.microsoft.com/mobileapps.Lakehouse/Tables/att/mobile_applications_partitioned"
MONGODB_DATABASE = "mobile_apps_0126"
MONGODB_COLLECTION = "applications_unsharded_1m"
conn_str = ""

In [ ]:
# Import libraries
from pyspark.sql.functions import *

print("✅ Libraries imported")

In [ ]:
# Read data from Delta Lake
delta_df = spark.read.format("delta").load(DELTA_TABLE_PATH)
record_count = delta_df.count()

print(f"✅ Loaded {record_count:,} records")
delta_df.select("_id", "fields.state", "fields.deviceManufacturer").show(5, truncate=False)

In [ ]:
# Auto-adjust batch size based on record count
if record_count <= 20000000:
    BATCH_SIZE = "2000"
elif record_count <= 50000000:
    BATCH_SIZE = "5000"
elif record_count <= 100000000:
    BATCH_SIZE = "8000"
else:
    BATCH_SIZE = "10000"

print(f"📦 Batch size: {BATCH_SIZE}")

In [ ]:
# Insert data into Azure DocumentDB
if not conn_str:
    raise ValueError("❌ Connection string not set")

print(f"🚀 Inserting {record_count:,} records...")

delta_df.write.format("mongodb") \
    .mode("append") \
    .option('connection.uri', conn_str) \
    .option("database", MONGODB_DATABASE) \
    .option("collection", MONGODB_COLLECTION) \
    .option("convertJson", "true") \
    .option("idFieldList", "_id,fields.state") \
    .option("ignoreNullValues", "true") \
    .option("maxBatchSize", BATCH_SIZE) \
    .option("operationType", "update") \
    .option("ordered", "true") \
    .option("upsertDocument", "true") \
    .save()

print(f"✅ Inserted {record_count:,} records")
print(f"🗄️  Database: {MONGODB_DATABASE}.{MONGODB_COLLECTION}")